In [3]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth
import os
# get the client id and client secret from the text file
with open('spotify_credentials.txt') as f:
    client_id = f.readline().strip().split(' ')[1]
    client_secret = f.readline().strip().split(' ')[1]
    uri = f.readline().split(' ')[1][:-1]

uri = 'http://localhost:8000'

# set the environment variables
os.environ['SPOTIPY_CLIENT_ID'] = client_id
os.environ['SPOTIPY_CLIENT_SECRET'] = client_secret
os.environ['SPOTIPY_REDIRECT_URI'] = uri
#sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=client_id,
                                               client_secret=client_secret,
                                            #   redirect_uri=uri,
                                               scope="user-library-read"))

scope = 'user-library-read'
username = '1260351083'

token = spotipy.util.prompt_for_user_token(username, scope)

if token:
    spotipy_obj = spotipy.Spotify(auth=token)
    saved_tracks_resp = spotipy_obj.current_user_saved_tracks(limit=50)
else:
    print('Couldn\'t get token for that username')

In [7]:
results = sp.search(q='weezer', limit=20)

Using `localhost` as redirect URI without a port. Specify a port (e.g. `localhost:8080`) to allow automatic retrieval of authentication code instead of having to copy and paste the URL your browser is redirected to.


In [27]:
results['tracks']['items'][0]['name']

'Island In The Sun'

In [28]:
hp = 'spotify:playlist:1nNxmkhYsOpa6Ux9MFJLoc?si=0f6e01db9bf348b9'

In [29]:
offset = 0

while True:
    response = sp.playlist_items(hp,
                                 offset=offset,
                                 fields='items.track.id,total',
                                 additional_types=['track'])

SpotifyException: http status: 400, code:-1 - Unsupported URL / URI., reason: None